# Downloaders

> Downloaders are responsible for downloading evaluation repositories from various sources (e.g. IOM, UNHCR).

In [ ]:
#| default_exp downloaders

In [ ]:
#| export
from pathlib import Path
from rich import print
from fastcore.all import *
from urllib.parse import urlparse
import requests
from functools import partial

from evaluatr.readers import load_evals, default_config

In [ ]:
#| exports
def flatten_evals_id_doc(
    evals: L,  # List of evaluation records
    cfg: dict = default_config  # Config dict with field names
) -> L:  # Returns list of (id, url) tuples
    "Flatten evaluation records into list of (id, url) tuples"
    return L((eval_data[cfg['id_field']], doc[cfg['url_field']]) 
             for eval_data in evals 
             for doc in eval_data[cfg['docs_field']])

In [ ]:
#| exports
def flatten_evals_id_doc(evals:L, cfg:dict=default_config) -> L:
    "Flatten evaluation records into list of (id, url) tuples"
    id,url,docs = cfg['id_field'],cfg['url_field'],cfg['docs_field']
    #return L((getattr(evl, id), doc[url]) for evl in evals for doc in getattr(evl, docs))
    for evl in evals:
        for doc in getattr(evl, docs):
            print(doc)
            break
    
    #return L((getattr(evl, id), doc[url]) for evl in evals for doc in getattr(evl, docs))

In [ ]:
fname = 'files/test/evaluations.json'
evals = load_evals(fname)
cfg = default_config
id,url,docs = cfg['id_field'],cfg['url_field'],cfg['docs_field']
L((getattr(evl, id), d['url']) for evl in evals[:2] for d in evl.docs)

(#5) [('9992310969aa2f428bc8aba29f865cf3', 'https://evaluation.iom.int/sites/g/files/tmzbdl151/files/docs/resources/Annex%20VI%20Case%20Study%20-%20RDH%20East%2C%20Horn%20and%20Southern%20Africa.pdf'),('9992310969aa2f428bc8aba29f865cf3', 'https://evaluation.iom.int/sites/g/files/tmzbdl151/files/docs/resources/Annex%20VII%20Case%20Study%20-%20RDH%20Asia-Pacific.pdf'),('2b297bc904ed1e542fbe67f62a8ea7cf', 'https://evaluation.iom.int/sites/g/files/tmzbdl151/files/docs/resources/Meta-Evaluation-Infographics.pdf'),('2b297bc904ed1e542fbe67f62a8ea7cf', 'https://evaluation.iom.int/sites/g/files/tmzbdl151/files/docs/resources/Meta-evaluation%202020-2024%20-%20Evaluation%20Brief_0.pdf'),('2b297bc904ed1e542fbe67f62a8ea7cf', 'https://evaluation.iom.int/sites/g/files/tmzbdl151/files/docs/resources/Meta-evaluation%202020-2024%20report%20-%20clean.pdf')]

In [ ]:
#| exports
def flatten_evals_id_doc(evals:L, cfg:dict=default_config) -> L:
    "Flatten evaluation records into list of (id, url) tuples"
    id,url,docs = cfg['id_field'],cfg['url_field'],cfg['docs_field']
    return L((getattr(evl, id), d[url]) for evl in evals for d in getattr(evl, docs))

In [ ]:
#| eval: false
fname = 'files/test/evaluations.json'
docs_to_download = flatten_evals_id_doc(load_evals(fname))
docs_to_download[:5]

KeyError: 'File URL'

I am importing this dict from the "readers" module, to specify what are the standardized fields for our .json file.

```
default_config = {
    'id_field': 'id',
    'docs_field': 'docs', 
    'url_field': 'File URL'
}
```

Apart from that "url_field" is actually wrong (should be 'url'), how would fastcore approach this?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

In fastcore style, you'd use `AttrDict` to make dict keys accessible as attributes, which is cleaner and more concise:

```python
default_config = AttrDict(id_field='id', docs_field='docs', url_field='url')
```

Then you can access fields as `cfg.id_field` instead of `cfg['id_field']`, and your function becomes:

```python
def flatten_evals_id_doc(evals:L, cfg=default_config) -> L:
    "Flatten evaluation records into list of (id, url) tuples"
    return L((getattr(evl, cfg.id_field), d[cfg.url_field]) for evl in evals for d in getattr(evl, cfg.docs_field))
```

I don't understand why it fails here?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

The issue is that `default_config` has `'url_field': 'File URL'`, but your actual data uses `'url'` as the key (as you can see from the working example in cell `_e5ea4e69` where you used `d['url']`).

You have two options:

1. Fix the config to match your data:
```python
cfg = default_config | {'url_field': 'url'}
docs_to_download = flatten_evals_id_doc(load_evals(fname), cfg)
```

2. Or check what keys are actually in your documents:
```python
evl = load_evals(fname)[0]
evl.docs[0].keys()
```

In [ ]:
#| exports
def mk_dirs(
    docs_to_download: L, # list of (eval_id, url) tuples
    base_dir: Path # path to the base directory
    ) -> None:
    "Create directories for each unique evaluation ID"
    unique_eval_ids = set(eval_id for eval_id, url in docs_to_download)
    for eval_id in unique_eval_ids:
        (base_dir / eval_id).mkdir(parents=True, exist_ok=True)

In [ ]:
#| eval: false
base_dir = Path("files/test/pdf_library")
mk_dirs(docs_to_download[:2], base_dir)

In [ ]:
#| exports
def extract_fname(url: str) -> str: 
    parsed_url = urlparse(url)
    fname = Path(parsed_url.path).name
    return fname

In [ ]:
#| exports
def download_doc(
    doc_info: tuple[str, str],  # (eval_id, url) for the document to download
    base_dir: Path, # Base directory to save files to
    overwrite: bool = True # If True, overwrite existing file
) -> str:
    "Download a document from a given URL and save it to a specified directory."
    eval_id, url = doc_info    
    
    fname = extract_fname(url)
    path = base_dir / eval_id / fname
    
    if path.exists() and not overwrite: 
        return f"Skipped {fname} (already exists)"
    
    try:
        r = requests.get(url, timeout=15)
        r.raise_for_status()
        
        with open(path, 'wb') as f:
            f.write(r.content)
            
        return f"Downloaded {fname}"
    except Exception as e:
        return f"Failed to download {fname}: {e}"

In [ ]:
#| eval: false
docs_to_download = flatten_evals_id_doc(load_evals(fname))
print(docs_to_download[:2])

[('1a57974ab89d7280988aa6b706147ce1', 
'https://evaluation.iom.int/sites/g/files/tmzbdl151/files/docs/resources/Internal%20Evaluation_NG20P0516_MAY_2023_F
INAL_Abderrahim%20EL%20MOULAT.pdf'), ('1a57974ab89d7280988aa6b706147ce1', 
'https://evaluation.iom.int/sites/g/files/tmzbdl151/files/docs/resources/RR0163_Evaluation%20Brief_MAY_%202023_Abde
rrahim%20EL%20MOULAT.pdf')]

In [ ]:
#| eval: false
eval_id, url = docs_to_download[0]
download_doc((eval_id, url), base_dir=base_dir, overwrite=False)

'Downloaded Internal%20Evaluation_NG20P0516_MAY_2023_FINAL_Abderrahim%20EL%20MOULAT.pdf'

In [ ]:
#| exports
def download_docs(
    json_file: str, # path to the JSON file
    base_dir: Path = Path("./PDF_Library"), # path to the base directory
    n_workers: int = 5, # number of workers
    overwrite: bool = False, # if True, overwrite existing files
    cfg: dict = default_config # config for the JSON file
) -> list:
    "Download all evaluation documents from a JSON file in parallel"
    docs_to_download = flatten_evals_id_doc(load_evals(json_file))
    mk_dirs(docs_to_download, base_dir)
    download_func = partial(download_doc, base_dir=base_dir, overwrite=overwrite)
    results = parallel(
        download_func, 
        docs_to_download, 
        n_workers=n_workers,
        total=len(docs_to_download),
        progress=True
        )
    
    return results

In [ ]:
#| eval: false
fname = 'files/test/evaluations.json'
base_dir = Path("files/test/pdf_library")
download_docs(fname, base_dir=base_dir, n_workers=0, overwrite=True)

(#24) ['Downloaded Internal%20Evaluation_NG20P0516_MAY_2023_FINAL_Abderrahim%20EL%20MOULAT.pdf','Downloaded RR0163_Evaluation%20Brief_MAY_%202023_Abderrahim%20EL%20MOULAT.pdf','Downloaded IB0238_Evaluation%20Brief_FEB_%202023_Abderrahim%20EL%20MOULAT.pdf','Downloaded Internal%20Evaluation_IB0238__FEB_2023_FINAL%20RE_Abderrahim%20EL%20MOULAT.pdf','Downloaded IB0053_Evaluation%20Brief_SEP_%202022_Abderrahim%20EL%20MOULAT.pdf','Downloaded Internal%20Evaluation_IB0053_OCT_2022_FINAL_Abderrahim%20EL%20MOULAT_0.pdf','Downloaded Internal%20Evaluation_NC0030_JUNE_2022_FINAL_Abderrahim%20EL%20MOULAT_0.pdf','Downloaded NC0030_Evaluation%20Brief_June%202022_Abderrahim%20EL%20MOULAT.pdf','Downloaded CD0015_Evaluation%20Brief_May%202022_Abderrahim%20EL%20MOULAT.pdf','Downloaded Projet%20CD0015_Final%20Evaluation%20Report_May_202_Abderrahim%20EL%20MOULAT.pdf','Downloaded Internal%20Evaluation_Retour%20Vert_JUL_2021_Fina_Abderrahim%20EL%20MOULAT.pdf','Downloaded NC0012_Evaluation%20Brief_JUL%202021_A